In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Futures Valuation Workflow

Attributes
----------
futures
transaction types
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Futures Valuation Using OTE Transactions

This notebook shows a futures valuation workflow through the creation of two hypothetical futures contracts, one for coffee (Coffee 'C') and one for orange juice (FCOJ-A). These two futures contracts are booked into a new portfolio and valuations are run over 4 days to demonstrate the varying realised and unrealised profits and losses.

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as models

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Create API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(), app_name="futuresLoader"
)

In [3]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

In [4]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"

## 1. Create Portfolio

In [5]:
portfolio_code = "FuturesPortfolio"

try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[f"Transaction/{scope}/FutType"],
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id FuturesPortfolio because it already exists in scope ibor.


## 2. Create New Property

In [6]:
properties = [
    ("FutType", "string"),
]

In [7]:
for property_code, dtype in properties:
    try:
        properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain="Transaction",
                scope=scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code=dtype, scope="system"),
            )
        )
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Error creating Property Definition 'Transaction/ibor/FutType' because it already exists.


## 3. Create Futures

In [8]:
# Define function that creates futures
def create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    start_date,
    maturity_date,
    fut_identifier,
):
    ctc = models.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
    futuredef = models.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=100,
        underlying=models.ExoticInstrument(
            instrument_format=models.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        instrument_type="Future",
    )
    # persist the instrument
    futureDefinition = models.InstrumentDefinition(
        name=fut_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(fut_identifier)},
        definition=futuredef,
    )
    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(request_body=batchUpsertRequest)
    futLuid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print(futLuid)

### 3.1 Create Futures Contract 1 - FCOJ-A

In [9]:
# Variable definitions for use in create_futures_contract function call 1 (FCOJ-A)
start_date = datetime(2021, 1, 1, tzinfo=pytz.utc)
effectiveAt = datetime(2021, 1, 1, tzinfo=pytz.utc)
maturity_date = datetime(2021, 12, 31, tzinfo=pytz.utc)

dom_ccy = "USD"
convention = "ActualActual"
contract_code = "OJ"
contract_month = "H"
contract_size = 1000
country_id = "US"
fut_name = "FCOJ-A Futures"
exchange_code = "ICE"
exchange_name = "ICE"
ticker_step = 0.01
unit_value = 1
identifier = "FUT_ICEOJZDEC21"


# Create Futures Contract function call 1 (FCOJ-A)
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    unit_value,
    ticker_step,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D52


### 3.2 Create Futures Contract 2 - Coffee 'C'

In [10]:
# Variable definitions for use in create_futures_contract function call 2 (Coffee 'C')
start_date = datetime(2021, 1, 1, tzinfo=pytz.utc)
effectiveAt = datetime(2021, 1, 1, tzinfo=pytz.utc)
maturity_date = datetime(2021, 12, 31, tzinfo=pytz.utc)

dom_ccy = "USD"
convention = "ActualActual"
contract_code = "KC"
contract_month = "H"
contract_size = 1000
country_id = "US"
fut_name = "Coffee 'C' Futures"
exchange_code = "ICE"
exchange_name = "ICE"
ticker_step = 0.01
unit_value = 1
identifier = "FUT_ICEKCZDEC21"

# Create Futures Contract function call 2 (Coffee 'C')
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    unit_value,
    ticker_step,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D53


## 4. Transactions

### 4.1 Create a transaction configuration for buying futures

In [11]:
new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="BuyFut",
                description="An BuyFut transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[
                                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/FutType",
                        set_to=f"Future")
                ]),
                        models.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[
                                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/FutType",
                         map_from=f"Transaction/{scope}/FutType")
                ])
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

### 4.2 Create a transaction configuration for selling futures

In [12]:
new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="SellFut",
                description="An SellFut transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=-1,
                properties=None,
                mappings=[
                                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/FutType",
                        set_to=f"Future")
                ]),
                        models.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[
                                        models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/FutType",
                         map_from=f"Transaction/{scope}/FutType")
                ])
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

### 4.3 Book transactions

In [13]:
# Read in transaction data read from file
futures_transactions = pd.read_excel("data/futures_data.xlsx", sheet_name="transactions")

# Output transaction data
futures_transactions

,txn_id,txn_type,trade_date,quantity,client_id,price,contract_size,tc,FutType
0,fut_txn_001,BuyFut,2021-01-01T07:00:00Z,10,FUT_ICEKCZDEC21,100,1000,1000000,Synthetic cash for KC Future
1,fut_txn_002,BuyFut,2021-01-01T07:00:00Z,10,FUT_ICEOJZDEC21,100,1000,1000000,Synthetic cash for OJ Future
2,fut_txn_003,SellFut,2021-01-04T12:00:00Z,5,FUT_ICEOJZDEC21,106,1000,530000,Synthetic cash for OJ Future


In [14]:
for index, row in futures_transactions.iterrows():

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=portfolio_code,
        transaction_request=[
            models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers={
                    "Instrument/default/ClientInternal": row["client_id"]
                },
                transaction_date=row["trade_date"],
                settlement_date=row["trade_date"],
                units=row["quantity"],
                transaction_price=models.TransactionPrice(
                    price=row["price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["tc"], currency="USD"
                ),
                properties = {f"Transaction/{scope}/FutType": models.PerpetualProperty(
                        key=f"Transaction/{scope}/FutType",
                        value=models.PropertyValue(label_value=row["FutType"]),
                    )},
            )
        ],
    )

## 5. Quotes

In [15]:
# Read in futures data from file
futures_prices = pd.read_excel("data/futures_data.xlsx", sheet_name="prices")
# Output futures data
futures_prices

,date,prices,luid,future_type
0,2021-01-01T00:00:00Z,100,FUT_ICEKCZDEC21,Synthetic cash for OF Future
1,2021-01-01T12:00:00Z,101,FUT_ICEKCZDEC21,Synthetic cash for OF Future
2,2021-01-01T00:00:00Z,100,FUT_ICEOJZDEC21,Synthetic cash for KC Future
3,2021-01-02T00:00:00Z,98,FUT_ICEKCZDEC21,Synthetic cash for OF Future
4,2021-01-02T00:00:00Z,97,FUT_ICEOJZDEC21,Synthetic cash for KC Future
5,2021-01-03T00:00:00Z,101,FUT_ICEKCZDEC21,Synthetic cash for OF Future
6,2021-01-03T00:00:00Z,102,FUT_ICEOJZDEC21,Synthetic cash for KC Future
7,2021-01-04T00:00:00Z,103,FUT_ICEKCZDEC21,Synthetic cash for OF Future
8,2021-01-04T00:00:00Z,107,FUT_ICEOJZDEC21,Synthetic cash for KC Future


### 5.1 Book quotes

In [16]:
for index, row in futures_prices.iterrows():

    instrument_quotes = {
        "upsert_request_1": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["luid"],
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["date"],
            ),
            metric_value=models.MetricValue(value=row["prices"], unit="USD"),
        )
    }

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(
        scope=quotes_scope, request_body=instrument_quotes
    )

## 6. Valuations

### 6.1 Create valuation recipe

In [17]:
# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="futuresValuation",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="0D.5D",
            )
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="Lusid", credit="Lusid", equity="Lusid", fx="Lusid", rates="Lusid"
        ),
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=models.PricingContext(
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Future",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

### 6.2 Create daily valuation function

The get_daily_fut_val() function when given a date & time along with a portfolio returns a valuation of the held instruments with an indicator of their price relative to the agreed future price. This valuation therefore enables you to know if your futures are worth more or less than you agreed to pay for them at any given time.

In [18]:
def get_daily_fut_val(date, portfolio_code):

    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code="futuresValuation"),
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            models.AggregateSpec(f"Transaction/{scope}/FutType", "Value"),
            models.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Holding/default/Units", "Value"),
            models.AggregateSpec("Valuation/PV/Amount", "Value"),
            models.AggregateSpec("Valuation/PnL/UnrealisedMarket/PfolioCcy", "Value"),
            
        ],
        group_by=["Instrument/default/Name", f"Transaction/{scope}/FutType"],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)
    
    renam_cols = {
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Value",
            "Valuation/Exposure/Amount": "Exposure",
            "Transaction/ibor/FutType": "FutType",
            "Instrument/Definition/ContractSize": "ContractSize",
            "Quotes/Price": "Price",
            "Holding/default/Units": "Units",
            "Valuation/PnL/UnrealisedMarket/PfolioCcy": "UnrealisedP&L"}


    vals_df.rename(columns=renam_cols,

        inplace=True,
    )

    dftotal = vals_df['Value'].sum()
    new_row = {'Value': dftotal, '': "SUBTOTAL"}
    vals_df = vals_df.append(new_row, ignore_index=True)
    vals_df = vals_df.fillna("")
    
    columns_titles = ["","Value"] + list(renam_cols.values())
    vals_df=vals_df.reindex(columns=columns_titles)
    return vals_df

### 6.3 Daily Valuations

Note: The final row of each valuation table shows the subtotal of the values column and thus shows the combined profits/losses both realisesd and unrealised.

#### Day 1

At 9AM on Day 1 we enter into two new Futures contracts: OJ and Coffee. Throughout Day 1, the contract prices move. We then run a Valuation at end of Day 1. This valuation shows that the Coffee 'C' Future Contract position is "in the money" with an unrealised profit of $10,000.

In [19]:
get_daily_fut_val("2021-01-01T09:00:00Z", portfolio_code)

,,Value,InstrumentName,ClientInternal,Value,Exposure,FutType,ContractSize,Price,Units,UnrealisedP&L
0,,"1,000,000.00",Coffee 'C' Futures,FUT_ICEKCZDEC21,"1,000,000.00",nan,Future,"1,000.00",100.00,10.00,0.00
1,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for KC Future,1.00,,"-1,000,000.00",0.00
2,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for OJ Future,1.00,,"-1,000,000.00",0.00
3,,"1,000,000.00",FCOJ-A Futures,FUT_ICEOJZDEC21,"1,000,000.00",nan,Future,"1,000.00",100.00,10.00,0.00
4,SUBTOTAL,0.00,,,0.00,nan,,,,,


In [20]:
get_daily_fut_val("2021-01-01T18:00:00Z", portfolio_code)

,,Value,InstrumentName,ClientInternal,Value,Exposure,FutType,ContractSize,Price,Units,UnrealisedP&L
0,,"1,010,000.00",Coffee 'C' Futures,FUT_ICEKCZDEC21,"1,010,000.00",nan,Future,"1,000.00",101.00,10.00,"10,000.00"
1,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for KC Future,1.00,,"-1,000,000.00",0.00
2,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for OJ Future,1.00,,"-1,000,000.00",0.00
3,,"1,000,000.00",FCOJ-A Futures,FUT_ICEOJZDEC21,"1,000,000.00",nan,Future,"1,000.00",100.00,10.00,0.00
4,SUBTOTAL,"10,000.00",,,"10,000.00",nan,,,,,


#### Day 2

Throughout Day 2, the contract prices move again and we run the valuation again at the end of the day. This valuation shows that both the Coffee 'C' and FCOJ-A are both now "out of the money" with unrealised losses of \\$20,000 and \\$30,000 respectively.

In [21]:
get_daily_fut_val("2021-01-02T18:00:00Z", portfolio_code)

,,Value,InstrumentName,ClientInternal,Value,Exposure,FutType,ContractSize,Price,Units,UnrealisedP&L
0,,"980,000.00",Coffee 'C' Futures,FUT_ICEKCZDEC21,"980,000.00",nan,Future,"1,000.00",98.00,10.00,"-20,000.00"
1,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for KC Future,1.00,,"-1,000,000.00",0.00
2,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for OJ Future,1.00,,"-1,000,000.00",0.00
3,,"970,000.00",FCOJ-A Futures,FUT_ICEOJZDEC21,"970,000.00",nan,Future,"1,000.00",97.00,10.00,"-30,000.00"
4,SUBTOTAL,"-50,000.00",,,"-50,000.00",nan,,,,,


#### Day 3

Throughout Day 3, the contract prices move again and we run the valuation again at the end of the day. This valuation shows that both the Coffee 'C' and FCOJ-A have both returned to positions that are "in the money" with unrealised profits of \\$10,000 and \\$20,000 respectively.

In [22]:
get_daily_fut_val("2021-01-03T18:00:00Z", portfolio_code)

,,Value,InstrumentName,ClientInternal,Value,Exposure,FutType,ContractSize,Price,Units,UnrealisedP&L
0,,"1,010,000.00",Coffee 'C' Futures,FUT_ICEKCZDEC21,"1,010,000.00",nan,Future,"1,000.00",101.00,10.00,"10,000.00"
1,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for KC Future,1.00,,"-1,000,000.00",0.00
2,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for OJ Future,1.00,,"-1,000,000.00",0.00
3,,"1,020,000.00",FCOJ-A Futures,FUT_ICEOJZDEC21,"1,020,000.00",nan,Future,"1,000.00",102.00,10.00,"20,000.00"
4,SUBTOTAL,"30,000.00",,,"30,000.00",nan,,,,,


#### Day 4

Throughout Day 4, the contract prices move again and we run the valuation again at the end of the day. This valuation shows that both the Coffee 'C' and FCOJ-A have both remained in positions that are "in the money" with increased unrealised profits of \\$30,000 and \\$35,000 respectively (Not \\$70,000 as 5 units were sold). Due to the positive price movements in the last 24 hours, half of the FCOJ-A Units were sold at a value of 106 relative to the contracted future price resulting in \\$30,000 in realised profits.

In [23]:
get_daily_fut_val("2021-01-04T18:00:00Z", portfolio_code)

,,Value,InstrumentName,ClientInternal,Value,Exposure,FutType,ContractSize,Price,Units,UnrealisedP&L
0,,"1,030,000.00",Coffee 'C' Futures,FUT_ICEKCZDEC21,"1,030,000.00",nan,Future,"1,000.00",103.00,10.00,"30,000.00"
1,,"-1,000,000.00",USD,,"-1,000,000.00",nan,Synthetic cash for KC Future,1.00,,"-1,000,000.00",0.00
2,,"-470,000.00",USD,,"-470,000.00",nan,Synthetic cash for OJ Future,1.00,,"-470,000.00",0.00
3,,"535,000.00",FCOJ-A Futures,FUT_ICEOJZDEC21,"535,000.00",nan,Future,"1,000.00",107.00,5.00,"35,000.00"
4,SUBTOTAL,"95,000.00",,,"95,000.00",nan,,,,,
